In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.PosarMCParameters import *

In [38]:
data_date = "2018_01_12_14_42_33"
data_date_b = "2018_01_12_14_43_09"
data_dir = "/home/pleroy/DATA/2018_01_mesures_de_couplage_v1/" + data_date
data_dir_b = "/home/pleroy/DATA/2018_01_mesures_de_couplage_v1/" + data_date_b

In [39]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

In [40]:
samplesPerFile = params.rampsPerFile * params.samplesPerRamp

In [41]:
print(params.rampsPerFile * params.samplesPerRamp * 4)

36000000


In [50]:
loop = 0
filename = data_dir + '/record' + str(loop) + '.bin'
fd = open(filename,'rb')
record_a = np.fromfile(fd, dtype = np.int16, count = params.rampsPerFile * params.samplesPerRamp * 2)
record_a_0 = record_a[::2].reshape(int(params.rampsPerFile), params.samplesPerRamp)
record_a_1 = record_a[1::2].reshape(int(params.rampsPerFile), params.samplesPerRamp)

filename_b = data_dir_b + '/record' + str(loop) + '.bin'
fd_b = open(filename_b,'rb')
record_b = np.fromfile(fd_b, dtype = np.int16, count = params.rampsPerFile * params.samplesPerRamp * 2)
record_b_0 = record_b[::2].reshape(int(params.rampsPerFile), params.samplesPerRamp)
record_b_1 = record_b[1::2].reshape(int(params.rampsPerFile), params.samplesPerRamp)

In [51]:
data_reshaped = record_0_0.reshape(params.rampsPerFile, params.samplesPerRamp)
data_reshaped_b = record_b_0.reshape(params.rampsPerFile, params.samplesPerRamp)

In [52]:
scalingFactor = 2.4 / 16383
offset = 0

In [53]:
data_scaled = data_reshaped * scalingFactor + offset
data_scaled_b = data_reshaped_b * scalingFactor + offset

In [54]:
def two_scales(ax1, data1, data2, c1, c2):
    ax2 = ax1.twinx()

    ax1.plot(data1, color=c1)
    ax1.set_ylabel('raw')

    ax2.plot(data2, color=c2)
    ax2.set_ylabel('scaled')
    
    ax1.grid()
    return ax1, ax2

In [55]:
font = { 'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [56]:
nbSamplesPerHalfRamp = int(params.samplesPerRamp/2)

In [57]:
fig, ax = plt.subplots()
ax1, ax2 = two_scales(ax, data_reshaped_b[0,:], data_scaled_b[0,:], 'r', 'b')

In [71]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(data_reshaped[0,:], color='b', label='with metallic plate')
ax1.plot(data_reshaped_b[0,:], color='g', label='without plate')
ax1.set_ylabel('raw')
ax1.legend()

ax2.plot(data_scaled[0,:], color='b')
ax2.plot(data_scaled_b[0,:], color='g')
ax2.set_ylabel('scaled [V]')
    
ax1.grid()

In [87]:
firstRamp = data_reshaped[0, 0:nbSamplesPerHalfRamp]
secondRamp = data_reshaped[0, nbSamplesPerHalfRamp:2*nbSamplesPerHalfRamp]
firstRampScaled = firstRamp * scalingFactor + offset
secondRampScaled = secondRamp * scalingFactor + offset
fft = np.fft.fft(secondRampScaled)

In [88]:
plt.figure()
plt.plot(firstRampScaled, label=data_date)
plt.plot(secondRampScaled, label='second ramp '+data_date)
plt.legend()
plt.grid()

In [78]:
max_fft = 20 * np.log10( np.abs( np.max(np.abs(fft)) ) )
print("max fft = {:.2f} dB".format(max_fft))

max fft = 121.47 dB


In [89]:
plt.figure()
plt.plot(20*np.log10(np.abs(fft)), label=data_date)
plt.grid()
plt.legend()